In [1]:
import credentials
import settings

import re
import tweepy
import mysql.connector
import pandas as pd
from textblob import TextBlob

class MyStreamerListener(tweepy.StreamListener):
    def on_status(self, status):
        if status.retweeted:
            return False
        id_str = status.id_str
        created_at = status.created_at
        text = deEmojify(status.text)
        sentiment = TextBlob(status.text)
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count = status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude= status.coordinates['coordinates']
            latitude = status.coordinates['coordinates']
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        print(status.text)
        print("Long: {}, Lat: {}". format(longitude, latitude))
        
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location,user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql,val)
            mydb.commit()
            mycursor.close()


In [2]:
def clean_tweet(self, tweet): 

    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
                                
def deEmojify(text):
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [3]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="gaurav",
    database="TwitterDB",
    charset = 'utf8'
)

if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()


ProgrammingError: 1050 (42S01): Table 'facebook' already exists

In [4]:
auth  = tweepy.OAuthHandler(credentials.CONSUMER_KEY, credentials.CONSUMER_SECRET)
auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [5]:
t = True
while(t == True):
    myStreamerListener = MyStreamerListener()
    myStream = tweepy.Stream(auth = api.auth, listener = myStreamerListener)
    myStream.filter(languages=["en"], track = settings.TRACK_WORDS)
    mydb.close()

RT @davidmweissman: This is a private  group for former Trump supporters like myself &amp; it will keep trolls out so people can freely express…
Long: None, Lat: None
#oldskool #house #hardcore #oldskooledits https://t.co/SaQknxgkM5 Rhythmology (DJ Nipper Edit 2015) 320kbps MP3 by… https://t.co/6j4SF3AIbg
Long: None, Lat: None
You now have a claim to a stock like Apple, Ford, or Facebook. In order to keep this claim to your stock, sign up a… https://t.co/IAv2o5qfHY
Long: None, Lat: None
No not him too, can someone decode this one,sounds like hes talking to the cronies on house arrest Face with medica… https://t.co/IcKLvCC2li
Long: None, Lat: None
RT @kerpen: Has @GovernorVA  addressed the claims made here that he's been spending weekends at a beach house in Manteo, NC in violation of…
Long: None, Lat: None
RT @ThisWooly: This is HILARIOUS anyone out there like me and need a laugh?  #OneVoice1.                             https://t.co/KSvSQKECzM
Long: None, Lat: None
RT @maanreyes: IF O

MySQLInterfaceError: Python type list cannot be converted